## Bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.io import imread,imsave,imshow
from skimage.color import rgb2gray
from scipy.stats import norm, kurtosis, entropy, skew
from skimage.feature import graycoprops,graycomatrix
import cv2
from glob import glob
import os
import csv
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier as mlp
from sklearn.metrics import accuracy_score as acc
from sklearn.model_selection import cross_val_score # Cross Validation Function.
from sklearn.model_selection import KFold # KFold Class.
from sklearn.linear_model import LinearRegression # Linear Regression class.
from skimage.exposure import histogram
import tqdm
from sklearn.svm import SVC
import xgboost as xgb

## Classificadores

### MLP

In [2]:
def classification(X,y):

    """
    X é uma matriz onde cada linha corresponde aos atributos calculados com o descritor
    y são os rótulos da classe de cada imagem. Exemplo [0,1,0,0,1,2,2,0,0,0,...,1]
    """
    
    # separar conjuntos de treino e teste
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,shuffle=True, random_state=42)
    # criar instancia do classificador
    clf = mlp()
    # treinar o classificador
    clf.fit(X_train,y_train)
    # realizar a predicao
    pred = clf.predict(X_test)
    # calcular o resultado utilizando a acuracia
    acuracia = acc(y_test,pred)
    
    return acuracia

### Random Forest

In [3]:
def classificationForest(X,y):

    
    """ com random forest """

    # separar conjuntos de treino e teste
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.75,shuffle=True, random_state=42)
    # criar instancia do classificador
    clf = rfc()
    # treinar o classificador
    clf.fit(X_train,y_train)
    # realizar a predicao
    pred = clf.predict(X_test)
    # calcular o resultado utilizando a acuracia
    acuracia = acc(y_test,pred)
    


    return acuracia

### SVM

In [4]:
def classificationSVM(X, y):
    svc = SVC(gamma="auto")
    #cv_result = cross_val_score(svc, X, y, cv=10, scoring="accuracy")

    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,shuffle=True)
    # criar instancia do classificador
    clf = svc
    # treinar o classificador
    clf.fit(X_train,y_train)
    # realizar a predicao
    pred = clf.predict(X_test)
    # calcular o resultado utilizando a acuracia
    acuracia = acc(y_test,pred)


    return acuracia


### XBR

In [5]:
def classificationXBR(X, y):
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,shuffle=True)
    # criar instancia do classificador
    clf = xgb.XGBClassifier()
    # treinar o classificador
    clf.fit(X_train,y_train)
    # realizar a predicao
    pred = clf.predict(X_test)
    # calcular o resultado utilizando a acuracia
    acuracia = acc(y_test,pred)

    return acuracia
    

### Matriz

In [29]:
# cria um array normal com os dados: media, variancia, skewness, kurtosis, energy, entropy
def pegaMatrizAux(matriz):
    atual = 0
    cont = 733
    aux = []
    y = []
    for i in matriz.values:
        n = list(i)
        if n[10] == atual:
            aux.append([n[3], n[4], n[5], n[6], n[7], n[8]])
            y.append(n[10])
            cont -= 1
        
        if cont == 0:
            atual += 1
            cont = 733

    return aux, y

### Leitura

In [6]:
formulas = ['alongamento', 'logaritmo', 'potencia']

In [34]:
def LinhaCSV(linha, Arquivo='classificacao.csv'):
    # Ordem: formula,teste,mlp,randomForest,svm,xbr
    with open(Arquivo, 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(linha)

In [35]:
def pegaClassicacoes(matriz, y, f, teste):
    mlp = classification(matriz, y)
    forest = classificationForest(matriz, y)
    svm = classificationSVM(matriz, y)
    xbr = classificationXBR(matriz, y)
    linha = [f, teste, mlp, forest, svm, xbr]
    LinhaCSV(linha)

In [38]:
for i in glob('matrizes/*'):
    f = i.split('-')[1].split('.')[0]
    teste = i.split('-')[2].split('.')[0]
    arq = pd.read_csv(i)
    matriz, y = pegaMatrizAux(arq) # pega a matriz de atributos
    pegaClassicacoes(matriz, y, f, teste)